# CPI + CLI DATA COLLECTION


## OECD Composite Leading Indicator

In [ ]:
cli = obb.economy.composite_leading_indicator(start_date='2024-01-01', provider='oecd', country=['united_states'])
cli.to_polars()

## Consumer Price Index

### OECD Consumer Price Index

In [ ]:
oecd_cpi = obb.economy.cpi(start_date='2010-01-01', country='united_states', transform='yoy', provider='oecd',harmonized=False, expenditure='total')
oecd_cpi.results


### FRED Consumer Price Index

In [ ]:
fred_cpi = obb.economy.cpi(start_date='2024-07-01', country='united_states', transform='yoy', provider='fred')
fred_cpi.results

# humblCOMPASS 

In [10]:
%load_ext autoreload
%autoreload 2

from humbldata.toolbox.toolbox_controller import Toolbox

toolbox = Toolbox(
    # symbols=["AAPL"],
    # interval="1d",
    start_date="2020-01-01",
    end_date="2024-12-01",
    membership="humblPEON"
)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [11]:
compass = await toolbox.fundamental.humbl_compass(
    country="united_states",
    chart=True,
    recommendations=True
)



INFO: HumblCompassFetcher || START: fetch_data (async)
INFO: LogCacheHitPlugin || humbl_compass cache HIT & RETURNED
DEBUG: LogCacheHitPlugin || humbl_compass cache key: {"country": "united_states", "end_date": "2024-12-01", "start_date": "2022-12-02", "z_score": null}
INFO: HumblCompassFetcher || END: fetch_data (async) - Total time: 0.0190s


In [12]:
compass.to_polars()

date_month_start,country,cpi,cpi_3m_delta,cli,cli_3m_delta,humbl_regime
date,str,f64,f64,f64,f64,str
2022-12-01,"""United States""",6.45,-1.75,98.73,-0.3,"""humblBUST"""
2023-01-01,"""United States""",6.41,-1.34,98.69,-0.21,"""humblBUST"""
2023-02-01,"""United States""",6.04,-1.07,98.66,-0.14,"""humblBUST"""
2023-03-01,"""United States""",4.98,-1.47,98.64,-0.09,"""humblBUST"""
2023-04-01,"""United States""",4.93,-1.48,98.66,-0.03,"""humblBUST"""
…,…,…,…,…,…,…
2024-07-01,"""United States""",2.89,-0.46,99.74,0.06,"""humblBOOM"""
2024-08-01,"""United States""",2.53,-0.74,99.83,0.13,"""humblBOOM"""
2024-09-01,"""United States""",2.44,-0.53,99.96,0.26,"""humblBOOM"""


In [15]:
compass


HumblObject

id: 06827a4d-abc6-7d0d-8000-0fa9a30cccad
results: b"DSL_VERSION\x01\x00\x00\x00\x81\xadDataFrameScan\x82\xa2df\xc5\x0cX\xff\...
equity_data: None
provider: yfinance
warnings: [{'category': 'HumblDataWarning', 'message': 'Start date adjusted to 2022...
chart: [{'content': '{"data":[{"hovertemplate":"\\u003cb\\u003e%{text}\\u003c\\u002...
extra: {'humbl_regime_recommendations': {'asset_classes': {'best': [<AssetRecommend...
context_params: {'symbols': 'AAPL', 'interval': '1d', 'start_date': datetime.date(2...
command_params: {'country': 'united_states', 'cli_start_date': '2022-08-02', 'cpi_s...
backtest: <function Fundamental.humbl_compass.<locals>.backtest at 0x110dd18a0>

In [20]:
compass.show()

In [19]:
compass.warnings

[Warning_(category='HumblDataWarning', message='Start date adjusted to 2022-12-02 based on humblPEON membership (2Y of data).'),
 Warning_(category='HumblDataWarning', message='Z-score defaulted to None. No z-score data will be calculated.')]

In [8]:
from aiocache import RedisCache


await RedisCache(namespace="humbl_compass").clear()

True

In [4]:
%load_ext autoreload
%autoreload 2

import polars as pl
from humbldata.core.standard_models.openbbapi.EconomyCompositeLeadingIndicatorQueryParams import EconomyCompositeLeadingIndicatorQueryParams
from humbldata.core.utils.openbb_api_client import OpenBBAPIClient


cli_query_params = EconomyCompositeLeadingIndicatorQueryParams(
    start_date="2022-08-02",
    end_date="2024-11-01",
    provider="oecd",
    country="united_states",
)
api_client = OpenBBAPIClient()
cli_response = await api_client.fetch_data(
    obb_path="economy.composite_leading_indicator",
    api_query_params=cli_query_params,
)
oecd_cli_data = cli_response.to_polars(collect=False)
oecd_cli_data = oecd_cli_data.rename({"value": "cli"}).with_columns(
    [pl.col("date").dt.month_start().alias("date_month_start")]
)

INFO: OpenBBAPIClient || START: fetch_data (async)
INFO: OpenBBAPIClient || Prepared request for: http://127.0.0.1:6900/api/v1/economy/composite_leading_indicator?provider=oecd&start_date=2022-08-02&end_date=2024-11-01&country=united_states&adjustment=amplitude&growth_rate=false
INFO: OpenBBAPIClient || Fetching data from: http://127.0.0.1:6900/api/v1/economy/composite_leading_indicator?provider=oecd&start_date=2022-08-02&end_date=2024-11-01&country=united_states&adjustment=amplitude&growth_rate=false


INFO: OpenBBAPIClient || END: fetch_data (async) - Total time: 0.1862s


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
